In [1]:
import numpy as np
np.set_printoptions(precision=2)

In [2]:
def feature_encoder(obs, state_space_n):
    state = np.zeros((state_space_n, 1), dtype = float)
    state[obs] = 1.0
    return state

In [12]:
def LSTD(env, max_iteration, feature_encoder, policy, gamma, lambda_):
    A = 0
    b = 0
    state_visited = set()
    state_space_n = env.observation_space.n
    for itr in xrange(max_iteration):
        obs = env.reset()
        state = feature_encoder(obs, state_space_n)
        eligibility = state
        done = False
        state_visited = state_visited.union({obs})
        while not done:
            action = np.random.multinomial(1, policy[obs]).argmax()
            next_obs, reward, done, _ = env.step(action)
            next_state = feature_encoder(next_obs, state_space_n)
            A += eligibility.dot((state - gamma * next_state).T)
            b += eligibility*reward
            eligibility = lambda_*eligibility + next_state
            obs = next_obs
            state_visited = state_visited.union({next_obs})
    return A#, np.linalg.inv(A).dot(b)

In [4]:
from collections import defaultdict

In [5]:
tmp_policy = defaultdict(lambda: np.ones(4)/4.)

In [6]:
import gym
from gym.envs.registration import register, spec

MY_ENV_NAME='FrozenLakeNonSlippery-v0'
try:
    spec(MY_ENV_NAME)
except:
    register(
        id=MY_ENV_NAME,
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name': '4x4', 'is_slippery': False},
    )
FLenv = gym.make(MY_ENV_NAME)

INFO:gym.envs.registration:Making new env: FrozenLakeNonSlippery-v0
[2016-08-31 20:02:28,532] Making new env: FrozenLakeNonSlippery-v0


In [13]:
a = LSTD(FLenv, 20000, feature_encoder, tmp_policy, 1, 0.99)

In [19]:
(a != 0).sum(axis=1)

array([16, 16, 16, 16, 16,  0, 16,  0, 16, 16, 16,  0,  0, 16, 16,  0])

In [20]:
FLenv.render()

SFFF
FHFH
FFFH
HFFG
  (Right)
